# Achievement Gap in Berkeley
## Geographic Visualizations

In [1]:
import pandas as pd
import numpy as np
import folium
from folium.plugins import MarkerCluster
import zipfile

%matplotlib inline

In [2]:
#Using Keilyn's cleaned 2018 caaspp data
clean_data = pd.read_csv("data/ky_caaspp_2018_clean.csv")
#File containing Berkeley public schools with geographic coordinates
school_data = pd.read_csv("data/2018berkschools.csv").drop('zipcode', axis=1)

crs = {'init': 'epsg:4326'}

#Exlcuding private and charter schools
non_private = clean_data[clean_data['school'].isin(school_data['school'])]
public_data = non_private[non_private['school'] != 'Berkeley Unified District Level Program']
public_data.head()

,Unnamed: 0,county_code_x,district_code_x,school_code_x,filler,year,subgroup_id,type,total_tested,total_tested_scores,...,category,subgroup,county_code_y,district_code_y,school_code_y,type_id,county,district,school,zipcode
28,28,1,61143,131177,NaN,2018,1,B,571,570,...,All Students,All Students,1.0,61143.0,131177.0,7.0,Alameda,Berkeley Unified,Berkeley High,94704.0
29,29,1,61143,131177,NaN,2018,1,B,604,600,...,All Students,All Students,1.0,61143.0,131177.0,7.0,Alameda,Berkeley Unified,Berkeley High,94704.0
30,30,1,61143,131177,NaN,2018,1,B,604,600,...,All Students,All Students,1.0,61143.0,131177.0,7.0,Alameda,Berkeley Unified,Berkeley High,94704.0
31,31,1,61143,131177,NaN,2018,1,B,571,570,...,All Students,All Students,1.0,61143.0,131177.0,7.0,Alameda,Berkeley Unified,Berkeley High,94704.0
48,48,1,61143,6056857,NaN,2018,1,B,961,955,...,All Students,All Students,1.0,61143.0,6056857.0,7.0,Alameda,Berkeley Unified,Martin Luther King Middle,94703.0


**Consolidating scores that did not meet standards**

In [3]:
nearly_met = public_data.loc[:, 'pct_nearly_met'].fillna(0)
not_met = public_data.loc[:,'pct_not_met'].fillna(0)
public_data['pct_not_met_total'] =  sum([nearly_met, not_met])

In [4]:
def single_test_mark(data, map1):
    mc = MarkerCluster()
    
    
    for r in np.arange(0, len(data)):
        if data['pct_met_above'][r] >= 50:
            mc.add_child(folium.Marker(
            location=[data['long'][r], data['lat'][r]],
            popup=folium.Popup(data['school'][r] + ", Grade: " + str(data['grade'][r]) + ", Percent Met and Above: " + str(data['pct_met_above'][r]), max_width=300,min_width=300),
            icon=folium.Icon(color='green')
            ))
            
    for r in np.arange(0, len(data)):
        if data['pct_met_above'][r] < 50:
            mc.add_child(folium.Marker(
            location=[data['long'][r], data['lat'][r]],
            popup=folium.Popup(data['school'][r] + ", Grade: " + str(data['grade'][r]) + ", Percent Met and Above: " + str(data['pct_met_above'][r]), max_width=300,min_width=300),
            icon=folium.Icon(color='red')
            ))     
    map1.add_child(mc) 
    return map1

def both_test_mark(data, map1):
    mc = MarkerCluster()

    for r in np.arange(0, len(data)):
        if data['test_id'][r] == 1:
            mc.add_child(folium.Marker(
                location=[data['long'][r], data['lat'][r]],
                popup=data['school'][r] + " Grade: " + str(data['grade'][r]) + " Percent Met and Above " + str(data['pct_met_above'][r]),
                icon=folium.Icon(color='orange')
            ))

    for r in np.arange(0, len(data)):
        if data['test_id'][r] == 2:
            mc.add_child(folium.Marker(
                location=[data['long'][r], data['lat'][r]],
                popup=data['school'][r] + " Grade: " + str(data['grade'][r]) + " Percent Met and Above " + str(data['pct_met_above'][r]),
                icon=folium.Icon(color='blue')
            ))          

    map1.add_child(mc)
    return map1

# Non-SED Student

**Grabbing only data that has subgroup_id 111, non-Socially-Economically Disadvantaged Students**

In [5]:
non_sed = public_data.query("subgroup_id == 111")
all_non_sed = non_sed[['test_id','grade','pct_met_above', 'pct_not_met_total', 'zipcode', 'school']].merge(school_data.loc[:, "school":"lat"], how='inner', left_on='school', right_on='school').dropna()
all_non_sed.head()

,test_id,grade,pct_met_above,pct_not_met_total,zipcode,school,long,lat
0,1,11,77.01,22.98,94704.0,Berkeley High,37.868018,-122.27141
1,2,11,65.41,34.59,94704.0,Berkeley High,37.868018,-122.27141
2,2,13,65.41,34.59,94704.0,Berkeley High,37.868018,-122.27141
3,1,13,77.01,22.98,94704.0,Berkeley High,37.868018,-122.27141
4,1,6,80.53,19.46,94703.0,Martin Luther King Middle,37.881804,-122.27702


# All Non-SED Results

In [6]:
all_non_sed_map = folium.Map(location=[school_data['long'].mean(),school_data['lat'].mean()], zoom_start=11.25)
single_test_mark(all_non_sed, all_non_sed_map)

# SED Student
**Grabbing all data that has subgroup_id 31, Economically Disadvantage Students.**

In [7]:
sed = public_data.query("subgroup_id == 31")

#Splitting data by test types: either ELA or Math.
sed_ela = sed.query("test_id == 1")
sed_math = sed.query("test_id == 2")

#Grabbing only scores
sed_both = sed[['test_id','grade','pct_met_above', 'pct_not_met_total', 'zipcode', 'school']].merge(school_data.loc[:, "school":"lat"], how='inner', left_on='school', right_on='school').dropna()
sed_ela_scores = sed_ela[['test_id', 'grade','pct_met_above', 'pct_not_met_total', 'zipcode', 'school']].merge(school_data.loc[:, "school":"lat"], how='inner', left_on='school', right_on='school').dropna()
sed_math_scores = sed_math[['grade','pct_met_above', 'pct_not_met_total', 'zipcode', 'school']].merge(school_data.loc[:, "school":"lat"], how='inner', left_on='school', right_on='school').dropna()

# Data from both tests for SED students

In [8]:
sed_both = sed_both.drop_duplicates(subset='pct_met_above').reset_index(drop=True)
sed_both.head()

,test_id,grade,pct_met_above,pct_not_met_total,zipcode,school,long,lat
0,1,11,44.94,55.06,94704.0,Berkeley High,37.868018,-122.27141
1,2,11,27.49,72.51,94704.0,Berkeley High,37.868018,-122.27141
2,1,6,40.26,59.74,94703.0,Martin Luther King Middle,37.881804,-122.27702
3,2,6,32.93,67.07,94703.0,Martin Luther King Middle,37.881804,-122.27702
4,2,7,43.33,56.66,94703.0,Martin Luther King Middle,37.881804,-122.27702


# All SED Results

In [9]:
all_sed_map = folium.Map(location=[school_data['long'].mean(),school_data['lat'].mean()], zoom_start=11.25)

single_test_mark(sed_both, all_sed_map)

## SED Elementary

In [10]:
elementary =school_data['school'].iloc[[3, 4, 5, 6, 7, 8, 10, 11, 12, 13]]
sed_both_elem = sed_both[sed_both['school'].isin(elementary)].reset_index(drop=True)
sed_both_5th = sed_both_elem.query("grade == 5").reset_index(drop=True)
sed_both_elem.head()

,test_id,grade,pct_met_above,pct_not_met_total,zipcode,school,long,lat
0,2,3,50.00,50.00,94710.0,Rosa Parks Environmental Science,37.865615,-122.29457
1,2,4,13.33,86.67,94710.0,Rosa Parks Environmental Science,37.865615,-122.29457
2,1,4,23.33,76.67,94710.0,Rosa Parks Environmental Science,37.865615,-122.29457
3,1,5,39.13,60.87,94710.0,Rosa Parks Environmental Science,37.865615,-122.29457
4,2,5,21.74,78.26,94710.0,Rosa Parks Environmental Science,37.865615,-122.29457


# SED Elementary Results

In [11]:
elem_map1 = folium.Map(location=[school_data['long'].mean(),school_data['lat'].mean()], zoom_start=12)

single_test_mark(sed_both_elem, elem_map1)

# Non-SED Elementary

In [12]:
non_sed_elem = all_non_sed[all_non_sed['school'].isin(elementary)].reset_index(drop=True)
non_sed_5th = non_sed_elem.query("grade == 5").reset_index(drop=True)
non_sed_elem.head()

,test_id,grade,pct_met_above,pct_not_met_total,zipcode,school,long,lat
0,2,3,95.83,4.17,94710.0,Rosa Parks Environmental Science,37.865615,-122.29457
1,1,3,95.83,4.17,94710.0,Rosa Parks Environmental Science,37.865615,-122.29457
2,1,4,80.85,19.15,94710.0,Rosa Parks Environmental Science,37.865615,-122.29457
3,2,4,78.72,21.28,94710.0,Rosa Parks Environmental Science,37.865615,-122.29457
4,2,5,72.55,27.45,94710.0,Rosa Parks Environmental Science,37.865615,-122.29457


## Non-SED Elementary Results

In [13]:
non_sed_elem_map1 = folium.Map(location=[school_data['long'].mean(),school_data['lat'].mean()], zoom_start=12)
single_test_mark(non_sed_elem, non_sed_elem_map1)

## SED 5th Grade Results

In [14]:
sed_5th_map = folium.Map(location=[school_data['long'].mean(),school_data['lat'].mean()], zoom_start=12)

single_test_mark(sed_both_5th, sed_5th_map)


## Non-SED 5th Grade Results

In [15]:
non_sed_5th_map = folium.Map(location=[school_data['long'].mean(),school_data['lat'].mean()], zoom_start=12)
single_test_mark(non_sed_5th, non_sed_5th_map)

# SED Middle Schools

In [16]:
middle_schools = school_data.iloc[[1, 2, 9 ]]
sed_both_mid = sed_both[sed_both['school'].isin(middle_schools['school'])].reset_index(drop=True)
sed_both_8th = sed_both_mid.query("grade == 8").reset_index(drop=True)
sed_8th_ela = sed_both_8th.query("test_id == 1").reset_index(drop=True)
sed_8th_math = sed_both_8th.query("test_id == 2").reset_index(drop=True)
sed_both_mid.head()

,test_id,grade,pct_met_above,pct_not_met_total,zipcode,school,long,lat
0,1,6,40.26,59.74,94703.0,Martin Luther King Middle,37.881804,-122.27702
1,2,6,32.93,67.07,94703.0,Martin Luther King Middle,37.881804,-122.27702
2,2,7,43.33,56.66,94703.0,Martin Luther King Middle,37.881804,-122.27702
3,1,7,49.44,50.56,94703.0,Martin Luther King Middle,37.881804,-122.27702
4,1,8,55.13,44.87,94703.0,Martin Luther King Middle,37.881804,-122.27702


## SED Middle School Results

In [17]:
sed_midmap1 = folium.Map(location=[school_data['long'].mean(),school_data['lat'].mean()], zoom_start=12)

single_test_mark(sed_both_mid, sed_midmap1)

# Non-SED Middle Schools

In [18]:
non_sed_mid = all_non_sed[all_non_sed['school'].isin(middle_schools['school'])].reset_index(drop=True)
non_sed_8th = non_sed_mid.query("grade == 8").reset_index(drop=True)
non_sed_mid.head()

,test_id,grade,pct_met_above,pct_not_met_total,zipcode,school,long,lat
0,1,6,80.53,19.46,94703.0,Martin Luther King Middle,37.881804,-122.27702
1,2,6,80.87,19.13,94703.0,Martin Luther King Middle,37.881804,-122.27702
2,2,7,79.10,20.90,94703.0,Martin Luther King Middle,37.881804,-122.27702
3,1,7,88.84,11.16,94703.0,Martin Luther King Middle,37.881804,-122.27702
4,1,8,88.07,11.93,94703.0,Martin Luther King Middle,37.881804,-122.27702


## Non-SED Middle School Results

In [19]:
non_sed_midmap1 = folium.Map(location=[school_data['long'].mean(),school_data['lat'].mean()], zoom_start=12)
single_test_mark(non_sed_mid, non_sed_midmap1)

## SED 8th Grade Results

In [22]:
sed_midmap2 = folium.Map(location=[school_data['long'].mean(),school_data['lat'].mean()], zoom_start=12)

single_test_mark(sed_both_8th, sed_midmap2)

## Non-SED 8th Grade Results

In [23]:
non_sed_midmap2 = folium.Map(location=[school_data['long'].mean(),school_data['lat'].mean()], zoom_start=12)
single_test_mark(non_sed_8th, non_sed_midmap2)

# SED High School

In [24]:
high = school_data.iloc[[0]]
sed_high = sed_both[sed_both['school'].isin(high['school'])].reset_index(drop=True)
sed_high

,test_id,grade,pct_met_above,pct_not_met_total,zipcode,school,long,lat
0,1,11,44.94,55.06,94704.0,Berkeley High,37.868018,-122.27141
1,2,11,27.49,72.51,94704.0,Berkeley High,37.868018,-122.27141


## SED High School Results
**Only have 11th grade data**

In [25]:
sed_high_map1 = folium.Map(location=[school_data['long'].mean(),school_data['lat'].mean()], zoom_start=12)
single_test_mark(sed_high, sed_high_map1)

# Non-SED High School 

In [26]:
non_sed_high = all_non_sed[all_non_sed['school'].isin(high['school'])].reset_index(drop=True)
non_sed_high

,test_id,grade,pct_met_above,pct_not_met_total,zipcode,school,long,lat
0,1,11,77.01,22.98,94704.0,Berkeley High,37.868018,-122.27141
1,2,11,65.41,34.59,94704.0,Berkeley High,37.868018,-122.27141
2,2,13,65.41,34.59,94704.0,Berkeley High,37.868018,-122.27141
3,1,13,77.01,22.98,94704.0,Berkeley High,37.868018,-122.27141


## Non-SED High School Results

In [27]:
non_sed_high_map1 = folium.Map(location=[school_data['long'].mean(),school_data['lat'].mean()], zoom_start=12)
single_test_mark(non_sed_high, non_sed_high_map1)